In [1]:
import pandas as pd
import numpy as np
import nltk


In [2]:
# annot_df = pd.read_csv('./coco-captions-2017-clean.csv', usecols=[1,2])
annot_df = None
with open('./annots/annotations/captions_train2017.json') as annot_file:
    captions_df = pd.read_json(annot_file, typ='series')
    annot_df = pd.DataFrame(data=captions_df['annotations'])
# annot_df = pd.read_csv('./resources/coco-captions-with-categories.csv', usecols=[0,1])

In [3]:
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings 
  
warnings.filterwarnings(action = 'ignore') 
  
import gensim 
from gensim.models import Word2Vec 
  

In [4]:
annot_list = annot_df.values
data = [] 
data_set = set()

# iterate through each caption 
for row in annot_list: 
    temp = []
    for j in word_tokenize(row[0]): 
        data_set.add(j.lower())
        temp.append(j.lower()) 

    data.append(temp) 
  


In [5]:
w2v_model = Word2Vec(data, size=100, workers=8, max_vocab_size=None, min_count = 0)

In [6]:
# w2v_model.save('./text_encoding.bin')
w2v_model.save('./resources/text_encoding_full.bin')

In [14]:
' '.join(word_tokenize('Bananas on display in a grocery store for $11.98/kilogram..'))

'Bananas on display in a grocery store for $ 11.98/kilogram..'